# Adım 1: Toplu Veri Yükleme (Bronze Layer)

Bu notebook, 'hotel_raw_batch.csv' dosyasını okuyup Fabric Lakehouse içerisine Delta tablosu olarak kaydeder.

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import current_timestamp, input_file_name

# Otomatik Spark Oturumu (Fabric)
spark = SparkSession.builder.getOrCreate()

# Dosya Yolu
SOURCE_PATH = "Files/data/hotel_raw_batch.csv" 
BRONZE_TABLE_NAME = "bronze_hotel_batch"

print(f"Okunuyor: {SOURCE_PATH}")

In [ ]:
# 1. Dosyayı Oku
df = spark.read.format("csv") \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .load(SOURCE_PATH)

# 2. Metadata Ekle
df_transformed = df.withColumn("ingestion_time", current_timestamp()) \
                   .withColumn("source_file", input_file_name())

# 3. Bronze Katmanına Yaz (Delta Table)
print(f"Tabloya yazılıyor: {BRONZE_TABLE_NAME}")

# HATA DUZELTME: overwriteSchema=true ile eski tablo semasini zorla degistiriyoruz
df_transformed.write.format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .saveAsTable(BRONZE_TABLE_NAME)

print("İşlem Başarılı! 'Tables' bölümünü yenileyerek tabloyu görebilirsiniz.")